# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat

In [2]:
# set the parameters to run the script
directories = [r'F:\polarimetry\measurements\HUMAN_EXPERIMENTS\FIXED_HUMAN\FIXED_HUMAN_EXPERIMENTAL\TheoniPics']
calib_directory = r'F:\polarimetry\calibration'

In [5]:
# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = True, parameter_set = 'TheoniPics', PDDN = False)

Traceback (most recent call last):
  File "C:\Users\romai\Documents\processingMM\src\processingmm\batch_processing.py", line 70, in find_folder_name
    assert len(re.findall("[\d]{4}-[\d]{2}-[\d]{2}", root)) == 1
AssertionError
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{}
{} 2022-01-10_T_Photo 4_FX_M_1
./temp_processing\2022-01-10_T_Photo 4_FX_M_1 2022-01-10 00:00:00
{}
{} 2022-01-10_T_Photo 4_FX_M_2
./temp_processing\2022-01-10_T_Photo 4_FX_M_2 2022-01-10 00:00:00


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [01:57<00:00, 117.02s/it]


['F:\\polarimetry\\measurements\\HUMAN_EXPERIMENTS\\FIXED_HUMAN\\FIXED_HUMAN_EXPERIMENTAL\\TheoniPics\\2022-01-10_T_Photo 4_FX_M_1',
 'F:\\polarimetry\\measurements\\HUMAN_EXPERIMENTS\\FIXED_HUMAN\\FIXED_HUMAN_EXPERIMENTAL\\TheoniPics\\2022-01-10_T_Photo 4_FX_M_2']